### Restart and Run All

In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 11, 19)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,TMT,12000,9.05,0,C9.0,SET,2022-08-24


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TMT,0.7000,0.0000,0.1500,0.0000,0.8500,"48,000",2022-08-24,2022-09-09,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'TMT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,645,TMT,15.00,9.70,S,7.90,9.05,13.30,1.01,9.90,"12,000",-5,11,"48,000",0,0,C9.0,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,B,BCH,3000,17.50,0,C21.46,SET100
1,O,CKP,20000,4.80,0,3L,SET100
2,O,CPF,4000,24.00,0,XXX,SET50
3,O,JMART,3000,42.00,0,1L,SET50
4,B,PTTGC,3000,41.25,0,5pct,SET50
5,B,RCL,3000,31.50,0,RD40pct,SET100
6,O,SCB,1000,102.00,0,XXX,SET50
7,B,STA,2500,18.10,0,5pct,SET100
8,O,SVI,12000,9.00,0,XXX,SET
9,B,SYNEX,3000,15.20,0,5pct,SET100


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,MCS,25000,9.70,0,5pct,SET
1,I,JASIF,10000,8.80,0,CP1S,SET
2,I,SCC,300,350.00,0,5pct,SET50
3,I,DOHOME,2400,14.20,0,5pct,SET100
4,S,KCE,1000,52.00,0,5pct,SET50
5,S,TMT,12000,9.05,0,C9.0,SET
6,S,ASP,10000,3.10,0,5pct,SET
7,I,BH,500,220.00,0,5pct,SET50
8,I,IVL,2400,44.25,0,5%,SET50


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(20, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,5,5,5,5,5,5
I,6,6,6,6,6,6
O,6,6,6,6,6,6
S,3,3,3,3,3,3


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,5,5,5,5,5,5
O,6,6,6,6,6,6
S,9,9,9,9,9,9


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(91, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(94, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
1,O,CKP,20000.0,4.8,0.0,3L,SET100,NaT,left_only
2,O,CPF,4000.0,24.0,0.0,XXX,SET50,NaT,left_only
10,O,TFG,20000.0,5.0,0.0,XXX,SET,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,5,5,5,5,5,5,5
O,6,6,6,6,6,6,6
S,9,9,9,9,9,9,9


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
1,O,CKP,20000,4.8,0,3L,SET100,NaT
2,O,CPF,4000,24.0,0,XXX,SET50,NaT
3,O,JMART,3000,42.0,0,1L,SET50,2022-04-19
6,O,SCB,1000,102.0,0,XXX,SET50,2022-02-14
8,O,SVI,12000,9.0,0,XXX,SET,2022-05-05
10,O,TFG,20000,5.0,0,XXX,SET,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
0,B,BCH,3000,17.50,0,C21.46,SET100,2022-08-31
4,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
5,B,RCL,3000,31.50,0,RD40pct,SET100,2022-11-24
7,B,STA,2500,18.10,0,5pct,SET100,2022-08-23
9,B,SYNEX,3000,15.20,0,5pct,SET100,2022-08-22


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
17,S,ASP,10000,3.10,0,5pct,SET,2022-08-24
18,S,BH,500,220.00,0,5pct,SET50,2023-03-11
14,S,DOHOME,2400,14.20,0,5pct,SET100,2022-02-02
19,S,IVL,2400,44.25,0,5%,SET50,2022-11-29
12,S,JASIF,10000,8.80,0,CP1S,SET,2022-11-21
15,S,KCE,1000,52.00,0,5pct,SET50,2022-08-22
11,S,MCS,25000,9.70,0,5pct,SET,2022-09-03
13,S,SCC,300,350.00,0,5pct,SET50,2022-08-10
16,S,TMT,12000,9.05,0,C9.0,SET,2022-08-24


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
0,B,BCH,3000,17.50,0,C21.46,SET100,2022-08-31
4,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
5,B,RCL,3000,31.50,0,RD40pct,SET100,2022-11-24
7,B,STA,2500,18.10,0,5pct,SET100,2022-08-23
9,B,SYNEX,3000,15.20,0,5pct,SET100,2022-08-22
1,O,CKP,20000,4.80,0,3L,SET100,NaT
2,O,CPF,4000,24.00,0,XXX,SET50,NaT
3,O,JMART,3000,42.00,0,1L,SET50,2022-04-19
6,O,SCB,1000,102.00,0,XXX,SET50,2022-02-14
8,O,SVI,12000,9.00,0,XXX,SET,2022-05-05


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,11,11,11,11,11,11,11
S,9,9,9,9,9,9,9


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

20

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'BCH', 3000, 17.5, 0, 'C21.46', 'SET100', '2022-08-31']
['B', 'PTTGC', 3000, 41.25, 0, '5pct', 'SET50', '2022-09-02']
['B', 'RCL', 3000, 31.5, 0, 'RD40pct', 'SET100', '2022-11-24']
['B', 'STA', 2500, 18.1, 0, '5pct', 'SET100', '2022-08-23']
['B', 'SYNEX', 3000, 15.2, 0, '5pct', 'SET100', '2022-08-22']
['B', 'CKP', 20000, 4.8, 0, '3L', 'SET100', 'NaT']
['B', 'CPF', 4000, 24.0, 0, 'XXX', 'SET50', 'NaT']
['B', 'JMART', 3000, 42.0, 0, '1L', 'SET50', '2022-04-19']
['B', 'SCB', 1000, 102.0, 0, 'XXX', 'SET50', '2022-02-14']
['B', 'SVI', 12000, 9.0, 0, 'XXX', 'SET', '2022-05-05']
['B', 'TFG', 20000, 5.0, 0, 'XXX', 'SET', 'NaT']
['S', 'MCS', 25000, 9.7, 0, '5pct', 'SET', '2022-09-03']
['S', 'JASIF', 10000, 8.8, 0, 'CP1S', 'SET', '2022-11-21']
['S', 'SCC', 300, 350.0, 0, '5pct', 'SET50', '2022-08-10']
['S', 'DOHOME', 2400, 14.2, 0, '5pct', 'SET100', '2022-02-02']
['S', 'KCE', 1000, 52.0, 0, '5pct', 'SET50', '2022-08-22']
['S', 'TMT', 12000, 9.05, 0, 'C9.0', 'SET', '2022-08-24']
['S', 'ASP'

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BCH,3000,17.50,0,C21.46,SET100,2022-08-31
1,2,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
2,3,B,RCL,3000,31.50,0,RD40pct,SET100,2022-11-24
3,4,B,STA,2500,18.10,0,5pct,SET100,2022-08-23
4,5,B,SYNEX,3000,15.20,0,5pct,SET100,2022-08-22
5,6,B,CKP,20000,4.80,0,3L,SET100,NaT
6,7,B,CPF,4000,24.00,0,XXX,SET50,NaT
7,8,B,JMART,3000,42.00,0,1L,SET50,2022-04-19
8,9,B,SCB,1000,102.00,0,XXX,SET50,2022-02-14
9,10,B,SVI,12000,9.00,0,XXX,SET,2022-05-05


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BCH,3000,17.50,0,C21.46,SET100,2022-08-31
1,2,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
2,3,B,RCL,3000,31.50,0,RD40pct,SET100,2022-11-24
3,4,B,STA,2500,18.10,0,5pct,SET100,2022-08-23
4,5,B,SYNEX,3000,15.20,0,5pct,SET100,2022-08-22
5,6,B,CKP,20000,4.80,0,3L,SET100,2022-02-02
6,7,B,CPF,4000,24.00,0,XXX,SET50,2022-02-02
7,8,B,JMART,3000,42.00,0,1L,SET50,2022-04-19
8,9,B,SCB,1000,102.00,0,XXX,SET50,2022-02-14
9,10,B,SVI,12000,9.00,0,XXX,SET,2022-05-05


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BCH,3000,17.50,0,C21.46,SET100,2022-08-31
1,2,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
2,3,B,RCL,3000,31.50,0,RD40pct,SET100,2022-11-24
3,4,B,STA,2500,18.10,0,5pct,SET100,2022-08-23
4,5,B,SYNEX,3000,15.20,0,5pct,SET100,2022-08-22
5,6,B,CKP,20000,4.80,0,3L,SET100,NaT
6,7,B,CPF,4000,24.00,0,XXX,SET50,NaT
7,8,B,JMART,3000,42.00,0,1L,SET50,2022-04-19
8,9,B,SCB,1000,102.00,0,XXX,SET50,2022-02-14
9,10,B,SVI,12000,9.00,0,XXX,SET,2022-05-05


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

3

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [33]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,7,B,CPF,4000,24.00,0,XXX,SET50,2022-02-02
1,8,B,JMART,3000,42.00,0,1L,SET50,2022-04-19
2,2,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
3,9,B,SCB,1000,102.00,0,XXX,SET50,2022-02-14
4,19,S,BH,500,220.00,0,5pct,SET50,2023-03-11
5,20,S,IVL,2400,44.25,0,5%,SET50,2022-11-29
6,16,S,KCE,1000,52.00,0,5pct,SET50,2022-08-22
7,14,S,SCC,300,350.00,0,5pct,SET50,2022-08-10


In [34]:
name = 'BLA'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.000000,20.00000,2.000000e+01,20.000000
mean,32.187500,32.725000,31.82500,5.611148e+06,32.300000
std,1.180642,1.126534,1.27759,3.960792e+06,1.163027
min,30.250000,30.750000,29.50000,2.214786e+06,30.000000
25%,30.937500,32.187500,30.43750,3.706500e+06,31.937500
50%,32.500000,33.250000,32.25000,4.498180e+06,32.625000
75%,33.062500,33.562500,32.75000,5.761554e+06,33.250000
max,33.750000,34.000000,33.25000,2.069161e+07,33.750000


### 2) Set100 records

In [35]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BCH,3000,17.5,0,C21.46,SET100,2022-08-31
1,6,B,CKP,20000,4.8,0,3L,SET100,2022-02-02
2,3,B,RCL,3000,31.5,0,RD40pct,SET100,2022-11-24
3,4,B,STA,2500,18.1,0,5pct,SET100,2022-08-23
4,5,B,SYNEX,3000,15.2,0,5pct,SET100,2022-08-22
5,15,S,DOHOME,2400,14.2,0,5pct,SET100,2022-02-02


In [36]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,70.816667,71.462500,70.254167,4.527611e+06,70.837500
std,1.497078,1.411083,1.524164,2.912959e+06,1.509732
min,66.750000,68.500000,66.500000,1.575220e+06,67.750000
25%,69.750000,70.500000,69.000000,3.002515e+06,69.500000
50%,70.875000,71.500000,70.375000,3.789318e+06,71.000000
75%,71.812500,72.500000,71.312500,4.914321e+06,72.000000
max,74.000000,74.000000,73.000000,2.134251e+07,74.000000


### 3) Set records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,10,B,SVI,12000,9.00,0,XXX,SET,2022-05-05
1,11,B,TFG,20000,5.00,0,XXX,SET,2022-02-02
2,18,S,ASP,10000,3.10,0,5pct,SET,2022-08-24
3,13,S,JASIF,10000,8.80,0,CP1S,SET,2022-11-21
4,12,S,MCS,25000,9.70,0,5pct,SET,2022-09-03
5,17,S,TMT,12000,9.05,0,C9.0,SET,2022-08-24


In [38]:
name = 'AIT'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,120.000000,120.000000,120.000000,1.200000e+02,120.000000
mean,6.220000,6.345000,6.130833,6.651949e+06,6.232083
std,0.437305,0.453715,0.418590,7.755256e+06,0.430838
min,5.500000,5.550000,5.400000,8.111610e+05,5.500000
25%,5.837500,5.987500,5.800000,2.091081e+06,5.850000
50%,6.200000,6.325000,6.100000,3.827604e+06,6.200000
75%,6.600000,6.700000,6.462500,7.618665e+06,6.600000
max,7.100000,7.300000,6.900000,4.831210e+07,7.100000
